In [7]:
import boto3
import sys
import pandas as pd
import os
import json
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import MaxNLocator
from scipy import stats

In [6]:
# connect to AWS client with saved key file
with open("aws_config/rootkey.csv","r") as kf:
    aws_info = kf.readlines()
    aws_ak =aws_info[0].split("=")[1].strip()
    aws_sk=aws_info[1].split("=")[1].strip()
client = boto3.client('iot', 
                      aws_access_key_id=aws_ak,
                      aws_secret_access_key=aws_sk, 
                      region_name='us-west-2')

In [18]:
# make a green grass object for each dataset
for i in range(0,5):
    #name them MyGreenGrass[i]
    tn = "MyGreenGrass"+str(i)
    print(tn)
    
    # make directory to save files if not already made
    new_dir = "Green_Grass_Config/device_{}/".format(i)
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)
    trResp= client.create_thing(thingName="MyGreenGrass"+str(i))
    crResp = client.create_keys_and_certificate(setAsActive=True)
    
    # make file names to save files to
    certf = "Green_Grass_Config/device_{}/device_{}_certificate.pem.crt".format(i,i)
    priv_keyf = "Green_Grass_Config/device_{}/device_{}_private.pem.key".format(i,i)
    prub_keyf = "Green_Grass_Config/device_{}/device_{}_public.pem.key".format(i,i)
    idf = "Green_Grass_Config/device_{}/device_{}_id.txt".format(i,i)
    
    # save certificates and keys
    with open(certf,"w") as crw:
        crw.write(crResp["certificatePem"])
    with open(priv_keyf, "w") as pkw:
        pkw.write(crResp["keyPair"]['PrivateKey'])
    with open(prub_keyf, "w") as prub2:
        prub2.write(crResp["keyPair"]['PublicKey'])
    with open(idf, "w") as idw:
        idw.write(crResp["certificateId"])
        
    # add communication and green grass policies to them
    apResp = client.attach_policy(policyName="Communication",
                             target = crResp["certificateArn"])
    apRespGreen = client.attach_policy(policyName="GreengrassV1CorePolicy",
                             target = crResp["certificateArn"])
    
    aprnResp = client.attach_thing_principal(thingName=trResp['thingName'],
                                         principal = crResp["certificateArn"])
    
    addgResp = client.add_thing_to_thing_group(thingGroupName= "TestThingGroup",
                                          thingGroupArn= "arn:aws:iot:us-west-2:298611685204:thinggroup/TestThingGroup",
                                          thingName =trResp["thingName"],
                                          thingArn = trResp["thingArn"])
    


MyGreenGrass0
MyGreenGrass1
MyGreenGrass2
MyGreenGrass3
MyGreenGrass4
